In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget https://raw.githubusercontent.com/google/aistplusplus_api/main/downloader.py

--2025-06-21 17:23:43--  https://raw.githubusercontent.com/google/aistplusplus_api/main/downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2744 (2.7K) [text/plain]
Saving to: ‘downloader.py’

downloader.py       100%[===================>]   2.68K  --.-KB/s    in 0s      

2025-06-21 17:23:44 (37.8 MB/s) - ‘downloader.py’ saved [2744/2744]



#2d points .json

##pkl load -> keypoints check

In [ ]:
import pickle
import os

pkl_path = '/content/drive/MyDrive/KMY/dataset/keypoints2d/gBR_sBM_cAll_d04_mBR0_ch01.pkl'

with open(pkl_path, 'rb') as f:
    data = pickle.load(f)

print(type(data))
print(data.keys() if isinstance(data, dict) else "Not a dict")

<class 'dict'>
dict_keys(['keypoints2d', 'det_scores', 'timestamps'])


In [ ]:
keypoints = data['keypoints2d']  # (T, N, 3)
print(keypoints.shape)

(9, 720, 17, 3)


##pkl to json

In [ ]:
# import pickle
# import numpy as np
# import os
# import json
# from tqdm import tqdm

# keypoint_dir = '/content/drive/MyDrive/KMY/dataset/keypoints2d'
# output_folder = '/content/drive/MyDrive/KMY/dataset/keypoints_json'
# os.makedirs(output_folder, exist_ok=True)

# pkl_files = sorted([f for f in os.listdir(keypoint_dir) if f.endswith('.pkl')])

# for pkl_file in tqdm(pkl_files):
#     pkl_path = os.path.join(keypoint_dir, pkl_file)

#     with open(pkl_path, 'rb') as f:
#         data = pickle.load(f)

#     keypoints = data['keypoints2d']  # shape: (T, 17, 3)

#     base_name = os.path.splitext(pkl_file)[0]
#     seq_folder = os.path.join(output_folder, base_name)
#     os.makedirs(seq_folder, exist_ok=True)

#     for i, frame in enumerate(keypoints):
#         pose_dict = {
#             "version": 1.3,
#             "people": [{
#                 "pose_keypoints_2d": frame[:, :2].flatten().tolist()
#             }]
#         }

#         out_path = os.path.join(seq_folder, f"{i:05d}_keypoints.json")
#         with open(out_path, 'w') as f:
#             json.dump(pose_dict, f)

100%|██████████| 1511/1511 [05:17<00:00,  4.76it/s]


In [ ]:
output_folder = '/content/drive/MyDrive/KMY/dataset/keypoints_json_test_100'
print(f"📁 Output folder: {output_folder}")

📁 Output folder: /content/drive/MyDrive/KMY/dataset/keypoints_json_test_100


In [ ]:
import pickle
import numpy as np
import os
import json
from tqdm import tqdm

# 경로 설정
keypoint_dir = '/content/drive/MyDrive/KMY/dataset/keypoints2d'
output_folder = '/content/drive/MyDrive/KMY/dataset/keypoints_json_test_100'
# os.makedirs(output_folder, exist_ok=True)

print(f"📁 Output folder: {output_folder}")

# PKL 파일 목록
pkl_files = sorted([f for f in os.listdir(keypoint_dir) if f.endswith('.pkl')])

# 파라미터
N_FRAMES = 120   # 추출할 프레임 수

for pkl_file in tqdm(pkl_files):
    pkl_path = os.path.join(keypoint_dir, pkl_file)

    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)

    keypoints = data['keypoints2d']      # (S, T, K, 3)
    det_scores = data['det_scores']      # (S, T)
    n_seq, n_frame, n_kp, _ = keypoints.shape

    # 1. 신뢰도 높은 시퀀스 1개 선택
    seq_idx = det_scores.mean(axis=1).argmax()
    best_kps = keypoints[seq_idx]        # (T, K, 3)

    # 2. 프레임 수 제한 (linspace로 균등 샘플링)
    if n_frame < N_FRAMES:
        print(f"⚠️ Warning: {pkl_file} has only {n_frame} frames. Skipping.")
        continue

    frame_indices = np.linspace(0, n_frame - 1, N_FRAMES).astype(int)
    sampled_frames = best_kps[frame_indices]  # (N_FRAMES, K, 3)

    # 3. JSON 저장 (폴더 X, 파일만 저장)
    base_name = os.path.splitext(pkl_file)[0]

    for i, frame in enumerate(sampled_frames):
        pose_dict = {
            "version": 1.3,
            "people": [{
                "pose_keypoints_2d": frame[:, :2].flatten().tolist()
            }]
        }

        filename = f"{base_name}_seq{seq_idx:02d}_frame{i:03d}_keypoints.json"
        out_path = os.path.join(output_folder, filename)

        with open(out_path, 'w') as f:
            json.dump(pose_dict, f)

📁 Output folder: /content/drive/MyDrive/KMY/dataset/keypoints_json_test_100


  0%|          | 2/1511 [00:03<38:40,  1.54s/it]


KeyboardInterrupt: 

#keypoints check


In [ ]:
import os, json

json_dir = '/content/drive/MyDrive/KMY/dataset/keypoints_json'
first_folder = sorted(os.listdir(json_dir))[0]
first_json_path = os.path.join(json_dir, first_folder, sorted(os.listdir(os.path.join(json_dir, first_folder)))[0])

with open(first_json_path) as f:
    data = json.load(f)

keypoints = data['people'][0]['pose_keypoints_2d']
num_points = len(keypoints) // 2
print(f"✅ {first_json_path} → 키포인트 수: {num_points}")


✅ /content/drive/MyDrive/KMY/dataset/keypoints_json/gBR_sBM_cAll_d04_mBR0_ch01/00000_keypoints.json → 키포인트 수: 2160


In [ ]:
import json

path = '/content/drive/MyDrive/KMY/dataset/keypoints_json/gBR_sBM_cAll_d04_mBR0_ch01/00000_keypoints.json'
with open(path) as f:
    data = json.load(f)

# 몇 명 사람?
print(f"사람 수: {len(data['people'])}")

# 첫 사람 keypoints 길이
kpts = data['people'][0]['pose_keypoints_2d']
print(f"Keypoint 길이: {len(kpts)} → 포인트 수: {len(kpts) // 2}")


사람 수: 1
Keypoint 길이: 4320 → 포인트 수: 2160


In [ ]:
import json

with open('/content/drive/MyDrive/KMY/dataset/keypoints_json/gBR_sBM_cAll_d04_mBR0_ch01/00000_keypoints.json') as f:
    d = json.load(f)

import pprint
pprint.pprint(d, depth=2)  # 전체 구조 확인

{'people': [{...}], 'version': 1.3}
